In [4]:
import pandas as pd
import os
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session



In [5]:
#read excel
fake_df = pd.read_excel('example_datatable.xlsx')

In [6]:
fake_df

,sample,age_gap,edu_gap,children,race_gap,homosexual
0,1,2,y,1.0,n,n
1,2,23,y,2.0,n,n
2,3,64,y,2.0,y,n
3,4,2,y,3.0,y,n
4,5,46,n,0.0,y,n
5,6,22,n,0.0,y,n
6,7,3,y,0.0,n,n
7,11,5,n,0.0,n,y
8,22,8,n,0.0,y,n
9,33,5,n,1.0,n,n


In [9]:
import re
import sqlite3

def get_create_table_string(tablename, connection):
    sql = """
    select * from sqlite_master where name = "{}" and type = "table"
    """.format(tablename) 
    result = connection.execute(sql)

    create_table_string = result.fetchmany()[0][4]
    return create_table_string

def add_pk_to_create_table_string(create_table_string, colname):
    regex = "(\n.+{}[^,]+)(,)".format(colname)
    return re.sub(regex, "\\1 PRIMARY KEY,",  create_table_string, count=1)

def add_pk_to_sqlite_table(tablename, index_column, connection):
    cts = get_create_table_string(tablename, connection)
    cts = add_pk_to_create_table_string(cts, index_column)
    template = """
    BEGIN TRANSACTION;
        ALTER TABLE {tablename} RENAME TO {tablename}_old_;

        {cts};

        INSERT INTO {tablename} SELECT * FROM {tablename}_old_;

        DROP TABLE {tablename}_old_;

    COMMIT TRANSACTION;
    """

    create_and_drop_sql = template.format(tablename = tablename, cts = cts)
    connection.executescript(create_and_drop_sql)

In [13]:
# # df = pd.DataFrame({"a": [1,2,3], "b": [2,3,4]})
# con = sqlite3.connect("deleteme.db")
# df.to_sql("df", con, if_exists="replace")

# add_pk_to_sqlite_table("df", "index", con)
# r = con.execute("select sql from sqlite_master where name = 'df' and type = 'table'")
# print(r.fetchone()[0])

con = sqlite3.connect("final_database.db")
fake_df.to_sql("df", con, if_exists="replace")

add_pk_to_sqlite_table("df", "index", con)
r = con.execute("select sql from sqlite_master where name = 'df' and type = 'table'")
print(r.fetchone()[0])


CREATE TABLE "df" (
"index" INTEGER PRIMARY KEY,
  "sample" INTEGER,
  "age_gap" INTEGER,
  "edu_gap" TEXT,
  "children" REAL,
  "race_gap" TEXT,
  "homosexual" TEXT
)


In [14]:
db = SQLAlchemy(app)

NameError: name 'SQLAlchemy' is not defined

In [ ]:
# import sqlite3
# import pandas as pd

# # Create the connection
# cnx = sqlite3.connect(r'C:\mydatabases\bigdata.db')

# # create the dataframe from a query
# df = pd.read_sql_query("SELECT * FROM userdata", cnx)